In [3]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Input,Flatten,Dense
from keras.utils.data_utils import get_file
import keras.backend as K
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, Input
import os
from os.path import join as pjoin
import h5py
import numpy as np
import tensorflow as tf


C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


encoder-decoder

In [2]:
MEAN_PIXEL = np.array([103.939, 116.779, 123.68])

def vgg_layers(inputs, target_layer):
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
#     x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)


    # Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)


    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(1, (3, 3), activation='relu', padding='same', name='downsample')(x)
    return x


def VGG19(input_tensor=None, input_shape=None, target_layer=1):
    """
    VGG19, up to the target layer (1 for relu1_1, 2 for relu2_1, etc.)
    """
    if input_tensor is None:
        inputs = Input(shape=input_shape)
    else:
        inputs = Input(tensor=input_tensor, shape=input_shape)
    model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19')
    # load_weights(model)
    return model


def preprocess_input(x):
    # Convert 'RGB' -> 'BGR'
    if type(x) is np.ndarray:
        x = x[..., ::-1]
    else:
        x = tf.reverse(x, [-1])

    return x - MEAN_PIXEL



from keras.layers import Input, Conv2D, UpSampling2D

def decoder_layers(inputs, layer):

    x=decoder_block4(inputs)
    x=decoder_block3(x)
    x=decoder_block2(x)
    x=decoder_block1(x)
    return x



def decoder_block1(inputs):
    x = UpSampling2D((2, 2), name='decoder_block1_upsample')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_block1_conv2')(x)
#     x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_block1_conv1')(x)
    return x

def decoder_block2(inputs):
    x = UpSampling2D((2, 2), name='decoder_block2_upsample')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block2_conv2')(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block2_conv1')(x)
    return x

def decoder_block3(inputs):
    x = UpSampling2D((2, 2), name='decoder_block3_upsample')(inputs)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block3_conv3')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block3_conv2')(x)
    return x

def decoder_block4(inputs):
    x = UpSampling2D((2, 2), name='decoder_block4_upsample')(inputs)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block4_conv3')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block4_conv1')(x)
    return x



In [3]:
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, Input
import keras.backend as K

LAMBDA=1

def l2_loss(x):
    return K.sum(K.square(x)) / 2

class EncoderDecoder:
    def __init__(self, input_shape=(256, 256, 3), target_layer=5,
                 decoder_path=None):
        self.input_shape = input_shape
        self.target_layer = target_layer

        self.encoder = VGG19(input_shape=input_shape, target_layer=target_layer)
        if decoder_path:
            self.decoder = load_model(decoder_path)
        else:
            self.decoder = self.create_decoder(target_layer)

        self.model = Sequential()
        self.model.add(self.encoder)
        self.model.add(self.decoder)

        self.loss = self.create_loss_fn(self.encoder)

        self.model.compile('adam', self.loss)

    def create_loss_fn(self, encoder):
        def get_encodings(inputs):
            encoder = VGG19(inputs, self.input_shape, self.target_layer)
            return encoder.output

        def loss(img_in, img_out):
            encoding_in = get_encodings(img_in)
            encoding_out = get_encodings(img_out)
            return l2_loss(img_out - img_in) + \
                   LAMBDA*l2_loss(encoding_out - encoding_in)
        return loss

    def create_decoder(self, target_layer):
        inputs = Input(shape=self.encoder.output_shape[1:])
        layers = decoder_layers(inputs, target_layer)
        output = Conv2D(3, (3, 3), activation='relu', padding='same',
                        name='decoder_out')(layers)
        return Model(inputs, output, name='decoder_%s' % target_layer)

    def export_decoder(self):
        self.decoder.save('decoder_%s.h5' % self.target_layer)

    def export_encoder(self):
        self.encoder.save('encoder_%s.h5' % self.target_layer)

In [4]:
encoder_decoder = EncoderDecoder(target_layer=3)
encoder_decoder.encoder.load_weights('encoder_3.h5')

In [5]:
Encoder=Sequential()
Encoder.add(encoder_decoder.encoder)
Encoder.add(Flatten())
Encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 16, 16, 1)         390721    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
Total params: 390,721
Trainable params: 390,721
Non-trainable params: 0
_________________________________________________________________


In [6]:
import matplotlib.image as mpimg
import cv2

In [7]:
# input a image then pass it to the model to get reconstruction
def get_reconstruction(model,img_path):
    newimg=mpimg.imread( img_path)
    #first we need to do the preprocessing
    # resize to vgg input size
    newimg=cv2.resize(newimg,(256,256))
    # add the batch dimension
    newimg = np.expand_dims(newimg, axis=0)
    # get the reconstruction 
    reconstruction_m1=model.predict(newimg)

    return reconstruction_m1

Already saved features. No need to run code below

In [8]:
# # get auto encoder feature for each video
# data_dir='C:\\NewResizedData'
# #data_dir contains 19 folders. Named as 1 2 3 4 5 .......
# for img_dir in os.listdir(data_dir):
#     # for each video
#     each_data=[]   
#     each_dir= pjoin(data_dir, img_dir)      #get each folder's name
#     print('Now reading......',each_dir,' there are',len(os.listdir(each_dir)),'images')
#     each_lable=img_dir
#     # read image in the folder
#     for i in range(1,len(os.listdir(each_dir))+1): 
        
#         img = each_dir+'\\'+ str(each_lable)+'-'+str(i)+'.jpg'

#         readimg=get_reconstruction(Encoder,img)# read each images and get feature
#         readimg= np.squeeze(readimg)  

#         each_data.append(readimg)
        
#     each_data=np.array(each_data)    
#     print(each_data.shape)
#     name='autoencoder_feature_'+ str(each_lable)+'.npy'
#     np.save(name, each_data) # save auto encoder feature
    

Now reading...... C:\NewResizedData\1  there are 739 images
(739, 256)
Now reading...... C:\NewResizedData\10  there are 968 images
(968, 256)
Now reading...... C:\NewResizedData\11  there are 753 images
(753, 256)
Now reading...... C:\NewResizedData\12  there are 731 images
(731, 256)
Now reading...... C:\NewResizedData\13  there are 640 images
(640, 256)
Now reading...... C:\NewResizedData\14  there are 731 images
(731, 256)
Now reading...... C:\NewResizedData\15  there are 486 images
(486, 256)
Now reading...... C:\NewResizedData\16  there are 737 images
(737, 256)
Now reading...... C:\NewResizedData\17  there are 735 images
(735, 256)
Now reading...... C:\NewResizedData\18  there are 734 images
(734, 256)
Now reading...... C:\NewResizedData\19  there are 678 images
(678, 256)
Now reading...... C:\NewResizedData\20  there are 615 images
(615, 256)
Now reading...... C:\NewResizedData\3  there are 808 images
(808, 256)
Now reading...... C:\NewResizedData\4  there are 739 images
(739, 

In [4]:
import pandas as pd
import numpy as np
# first we can see what does the feature look like
def get_raw_feature():
    video_data_dir='C:\\videofeature\\'
    X=[]
    Y=[]
    for video in os.listdir(video_data_dir):
        each_video= pjoin(video_data_dir,video)
        l=each_video.split('t_')[1]
        l=l.split('.')[0]
        LABEL=int(l)
        X.append(np.load(each_video))
        Y.append(LABEL)
    Video_label=pd.read_csv('label.csv')
    Video_label=np.array(Video_label)[:,:-1]
    return X,Y


In [5]:
X,Y=get_raw_feature()

In [6]:
auto_data_dir='C:\\auto_feature\\'
auto_X=[]
for video in os.listdir(auto_data_dir):
    each_video= pjoin(auto_data_dir,video)
    auto_X.append(np.load(each_video))

Finished saving combined_feature no need to run below

In [7]:
# # save combined feature
# for x, x_auto,y in zip(X,auto_X,Y):
#     combined_feature=np.hstack((x,x_auto))
    
#     name='combined_feature_'+ str(y)+'.npy'
#     np.save(name, combined_feature) # save auto encoder feature
